# Task 1.1 – Klassifisering: health_risk (Low/Medium/High)

## Mål (sensor-vennlig)
Målet er å bygge og evaluere flere klassifiseringsmodeller for å predikere `health_risk` (Low/Medium/High). Fokus er **ikke bare kode**, men:

- **begrunnede valg** (imputering/encoding/skalering/modellvalg)
- **systematisk evaluering** (confusion matrix, accuracy, macro F1)
- **feilanalyse** og konkrete forbedringsforslag

## Ord-budsjett (tips)
Hold teksten kort: problem → metode → resultater → feilanalyse → forbedringer.


## 1) Imports + config

In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
)
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 2) Load dataset

Legg datasettet ditt som CSV her (eksempel):
- `data/customer_health_profile.csv`

Bytt `DATA_PATH` om filnavnet ditt er annerledes.

In [ ]:
# TODO: legg inn riktig filsti
# Notebooken ligger i notebooks/, derfor bruker vi ../data/
DATA_PATH = Path("../data/customer_health_profile.csv")

if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Fant ikke {DATA_PATH}. Lag en data/-mappe ved siden av notebooken og legg inn CSV-en der."
    )

df = pd.read_csv(DATA_PATH)
df.head()

## 3) Quick EDA (kort og relevant)

- Datatyper + missing values
- Class balance for `health_risk`
- En kort kommentar: er datasettet balansert? hva betyr det for metrics?

In [ ]:
target_col = "health_risk"  # TODO: bytt hvis target heter noe annet

print("shape:", df.shape)
display(df.dtypes.value_counts())

missing = df.isna().mean().sort_values(ascending=False)
display(missing.head(20))

display(df[target_col].value_counts(dropna=False))
display(df[target_col].value_counts(normalize=True, dropna=False).rename("proportion"))

## 4) Split (train/test) + leakage-sjekk

Vi bruker **stratified split** siden dette er en klassifisering. Hold test-settet “rent” (ikke bruk det under tuning).


In [ ]:
X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y,
)

print("train:", X_train.shape, "test:", X_test.shape)
display(y_train.value_counts(normalize=True).rename("train_prop"))
display(y_test.value_counts(normalize=True).rename("test_prop"))

## 5) Preprocessing pipeline (imputering + encoding + scaling)

Sensor-fokus (skriv 5–8 setninger):
- **Imputering**: numerisk median, kategorisk mest-frekvent (robust + enkel baseline)
- **Encoding**: OneHot for kategoriske variabler
- **Scaling**: StandardScaler for modeller som er sensitive (SVM/KNN/LogReg)


In [ ]:
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

numeric_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

categorical_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ]
)

print("num_cols:", len(num_cols), "cat_cols:", len(cat_cols))
num_cols[:10], cat_cols[:10]

## 6) Tren og sammenlign flere modeller (baseline → best)

Vi sammenligner modeller med cross-validation (stratified). Rapportér **accuracy** og **macro F1**.


In [ ]:
models: Dict[str, object] = {
    "logreg": LogisticRegression(max_iter=2000),
    "svm_rbf": SVC(kernel="rbf"),
    "knn": KNeighborsClassifier(),
    # NB: GaussianNB kan ikke direkte ta sparse output fra OneHotEncoder.
    # Hvis du vil bruke NB, må vi enten gjøre onehot dense eller bruke BernoulliNB/MultinomialNB på passende features.
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

rows = []
for name, clf in models.items():
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", clf)])
    scores = cross_validate(
        pipe,
        X_train,
        y_train,
        cv=cv,
        scoring={"acc": "accuracy", "f1_macro": "f1_macro"},
        n_jobs=None,
        return_train_score=False,
    )
    rows.append(
        {
            "model": name,
            "acc_mean": float(np.mean(scores["test_acc"])),
            "acc_std": float(np.std(scores["test_acc"])),
            "f1_macro_mean": float(np.mean(scores["test_f1_macro"])),
            "f1_macro_std": float(np.std(scores["test_f1_macro"])),
        }
    )

results = pd.DataFrame(rows).sort_values(["f1_macro_mean", "acc_mean"], ascending=False)
results

## 7) Fit beste modell på train → eval på test

Her viser du:
- confusion matrix
- accuracy + macro F1
- kort feilanalyse (hvilke klasser blandes?)


In [ ]:
best_name = results.iloc[0]["model"]
best_clf = models[str(best_name)]
best_pipe = Pipeline(steps=[("preprocess", preprocess), ("model", best_clf)])

best_pipe.fit(X_train, y_train)
pred = best_pipe.predict(X_test)

acc = accuracy_score(y_test, pred)
f1m = f1_score(y_test, pred, average="macro")
print("best:", best_name)
print("accuracy:", acc)
print("macro_f1:", f1m)

print("\nclassification_report:\n", classification_report(y_test, pred))
confusion_matrix(y_test, pred)

## 8) Feilanalyse (konkret og kort)

Finn noen eksempler på feilklassifiseringer (f.eks. High→Medium) og beskriv mønsteret.


In [ ]:
errors = X_test.copy()
errors["y_true"] = y_test.values
errors["y_pred"] = pred

wrong = errors[errors["y_true"] != errors["y_pred"]]
print("wrong:", len(wrong), "/", len(errors))
wrong.head(10)

## 9) Refleksjon / forbedringer (sensor-points)

Skriv 8–12 linjer om:
- ubalanse i klasser → hvorfor macro F1 er relevant
- bedre imputering (IterativeImputer/KNNImputer) vs enkel baseline
- hyperparameter tuning (GridSearchCV) + hvorfor
- feature importance / forklarbarhet (permutation importance)
- begrensninger: datasettstørrelse, label-støy, generaliserbarhet
